In [0]:
# 02_transform_silver.py

from pyspark.sql.functions import when, col, trim, lower
from pyspark.sql.types import IntegerType
from tests.etl_pipeline import sjekk_duplikater, valider_manglende, valider_gyldige_verdier, konverter_timestamp

# Reload from bronze (independent of Notebook 01’s runtime)
df_raw = spark.table("default.staging_parking")

df_raw = df_raw.withColumn(
    "Antall_ledige_plasser",
    when(lower(trim(col("Antall_ledige_plasser"))) == "fullt", 0)
    .otherwise(col("Antall_ledige_plasser"))
    .cast(IntegerType())
)

valider_manglende(df_raw)
valider_gyldige_verdier(df_raw)
df_deduped = sjekk_duplikater(df_raw)
df_cleaned = konverter_timestamp(df_deduped)

silver_path = "/mnt/silver/parking_cleaned"

(
    df_cleaned.write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .save(silver_path)
)

spark.sql(f"""
CREATE TABLE IF NOT EXISTS default.parking_cleaned
USING DELTA
LOCATION '{silver_path}'
""")
